# Imports

In [ ]:
import logging
import platform
import sys
import os
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, Markdown
from datetime import datetime
# import tensorflow as tf

logger = logging.getLogger('_ipynb')
logger.setLevel(logging.DEBUG)


ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(levelname)s - %(asctime)s - %(name)s - %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)
logger.debug('--=logging started=--')

# print(tf.__version__)
CPU = platform.processor()
print (f'Running on CPU:{CPU}')


nb_dir = os.path.split(os.getcwd())[0]

if nb_dir not in sys.path:
    sys.path.append(nb_dir)

import analyser.hyperparams 
print(analyser.hyperparams.__file__)

### Prepare workdir

In [ ]:
# from pathlib import Path
# _work_dir_default = Path(analyser.hyperparams.__file__).parent.parent.parent / 'work'
# work_dir = os.environ.get('GPN_WORK_DIR', _work_dir_default)

# if not os.path.isdir(work_dir):
#     os.mkdir(work_dir)

from analyser.hyperparams import work_dir  
 
print('work_dir=', work_dir)
assert os.path.isdir(work_dir)

### Imports...

In [ ]:
%matplotlib inline

import pandas as pd
from pandas import DataFrame

from analyser.finalizer import get_doc_by_id
from analyser.persistence import DbJsonDoc
from integration.db import get_mongodb_connection
from pymongo import ASCENDING

### Import docs having orgs

In [ ]:
import re 

db = get_mongodb_connection()
documents_collection = db['documents']
sorting = [('analysis.analyze_timestamp', ASCENDING), ('user.updateDate', ASCENDING)]

query = {
  '$and': [
#     {"parse.documentType": "CONTRACT"},      
#     {"state": 15},
#     {"analysis.attributes_tree.contract.orgs.1.type": {"$type": 10}},
      
    {'$or': [
        {"user.attributes_tree.contract.orgs": {"$ne": None}},
        {"analysis.attributes_tree.contract.orgs": {"$ne": None}}
    ]}
  ]
}

query = {
    '$and': [
       {'$or': [
        {"user.attributes_tree.contract.orgs": {"$ne": None}},
        {"analysis.attributes_tree.contract.orgs": {"$ne": None}}
        ]},
        
        {'parse.paragraphs.0.paragraphHeader.text':{ "$regex":  re.compile(r'физическим лицом' , re.IGNORECASE)} }
    ]
}

res = documents_collection.find(filter=query, 
                                sort=sorting,
                                projection={'_id': True, 
                                            'user.updateDate':True, 
                                            "analysis.attributes_tree.contract.orgs":True,  
                                            "user.attributes_tree.contract.orgs":True,
                                           'parse.paragraphs.paragraphHeader.text':True}

                               ).limit(2000)



res_orgs = list([i for i in res])

_s = f"#### Всего документов с org  {len(res_orgs)}"
display(Markdown(_s))

# print(res_inside)

In [ ]:
# d['parse']['paragraphs'][0]['paragraphHeader']['text']

In [ ]:
res_orgs[0]

### Pull orgs from DB

In [ ]:
lines=[]
for k, obj in enumerate(res_orgs):
#     d = get_doc_by_id(oid)
#     jd = DbJsonDoc(d)
    tree = obj.get('user') or obj['analysis'] # jd.get_attributes_tree()
    tree = tree['attributes_tree'].get('contract',{})
#     print(tree)
    orgs = tree.get('orgs', [])
    
    
    line = [None]*6
    line[0] = oid
    line[-1] = obj['parse']['paragraphs'][0]['paragraphHeader']['text']
    
    for oi, o in enumerate(orgs):
#         print (o['type']['value'])
        line[ 1 + oi] = o.get('type', {}).get('value')
        line[ 3 + oi] = o.get('name', {}).get('value')
#     s = ins['span']
#     quote = doc[s[0]: s[1]].get_normal_text()
#     print(line)
    lines.append( line )
    
orgs = DataFrame(lines, columns=['uid', 'type_1','type_2', 'name_1', 'name_2', 'headline'])
orgs

In [ ]:
# orgs.__dict__
orgs[orgs.type_2.isna() | orgs.type_1.isna() ]

In [ ]:
from analyser.text_normalize import r_human_name
r_human_name

In [ ]:
r = re.compile(r'\W' + r_human_name, re.MULTILINE)

x = r.search('что-то Мироздания С.К., который был')
x['human_name']

In [ ]:
r = re.compile(r'\W' + r_human_name, re.MULTILINE)
for i, row in orgs.iterrows():
    x = r.search(str(row.name_2))
    if x is not None:
        print(x)